- Limpiar punctuacion
- Limpiar links

In [36]:
import pandas as pd
import nltk  
import numpy as np  
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout, GRU, Embedding, TimeDistributed, BatchNormalization, Bidirectional

import re
from collections import OrderedDict
import random
import os

In [2]:
df = pd.read_csv('todes.csv')

/opt/anaconda3/envs/Ironhack/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15,16,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 
## Tweet generation with LSTM
Example in this [link](https://towardsdatascience.com/tweet-generation-with-neural-networks-lstm-and-gpt-2-e163bfd3fbd8)
 1. **Removed tweets whose lengths were less than 60 characters**. 

    Removing tweets that fall under a certain length, it removes tweets that are one word and the empty tweets that made it in the CSV file
   
   
2.  **Tweets were divided into sequences of equal lengths and placed into a list called sentences.**
 
    These sentences will be used as training examples for the model.
 
 
 3. **Vectorization**
 
    Split the tweets into x and y evenly.**x** is a 3D matrix that has the shape of the total number of sentences, length of steps(40), and the number of unique characters. **y** is a 2D matrix that has the total length of sentences and unique characters and the purpose of this vector is to retrieve the next character that’s after a sentence given by x.
 
 
 4. **Model**

In [3]:
# model = Sequential()

# model.add(LSTM(len(chars) * 5, input_shape=(SEQUENCE_LENGTH, len(chars))))
# model.add(BatchNormalization())
# model.add(Activation('selu'))

# model.add(Dense(len(chars) * 2))
# model.add(BatchNormalization())
# model.add(Activation('selu'))

# model.add(Dense(len(chars) * 2))
# model.add(BatchNormalization())
# model.add(Activation('selu'))

# model.add(Dense(len(chars)))
# model.add(Activation('softmax'))

# optimizer = RMSprop(lr=0.001)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# model.fit(X, y, validation_split=0.05, batch_size=124, epochs=4, shuffle=True)

5. **Training the model**

      After 10 epoch the model stopped learning with a loss of around 1.5 for each of the datasets. During training, the model outputs the next character based on the probability distribution. This is known as diversity. Diversity is a value that determines how random the generated sentence is. For training purposes, we set the diversity to 0.2, 0.5, 0.7, 1.0, and 1.2 in order to visualize which diversity generated the most meaningful tweets.
  
  
6.  **Generate new tweets for the user**

    The generate_tweets function takes a random seed, which is a random tweet from the lists of tweets, and a set diversity. We chose a diversity of 0.7 because it formulated the most comprehensible tweets compared to the others. After receiving a random seed and a diversity value, the function cuts the seed length down to 40 characters. This is done because it would be benign to use an already formulated tweet to create another and because once a new tweet is generated, it would exceed the max length of characters that are permitted on twitter.

## ANALYSIS

### Dictionary to analyze the results

Clean tweets text

From https://towardsdatascience.com/predicting-trump-tweets-with-a-rnn-95e7c398b18e:

In [18]:
def removeLinks(text):
    return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)

def removeEmojis(text):
    return text.encode('ascii', 'ignore').decode('ascii')

def removeEllipsis(text):
    return re.sub('\.\.[\.]*', " ", text)

def removeParens(text):
    return re.sub("[\(\[].*?[\)\]]", "", text)

def removeLF(text):
    text = re.sub('\n',' ', text)
    text = re.sub(' [ ]*', ' ', text)
    return text

def removeSpecialCharacters(text):
    text = re.sub('[#@?¿.,;:!¡]',' ', text)
    return text

In [19]:
def df_to_clean_text(df):
    text = ''
    for tweet in df.full_text:
        text += tweet
    text = removeLinks(text)
    text = removeEllipsis(text)
    text = removeEmojis(text)
    text = removeParens(text)
    text = removeLF(text)
    text = removeSpecialCharacters(text)
    text = text.lower()
    print('Text lenght: ',len(text))
    return text

In [58]:
def word_freq(text,query,language):
    corpus = nltk.sent_tokenize(text)
    wordfreq = {}

    for sentence in corpus:
        
        tokens = nltk.word_tokenize(sentence)
        
        stop_words = set(nltk.corpus.stopwords.words(language))   
        filtered_sentence = [w for w in tokens if not w in stop_words]
        
        for token in filtered_sentence:
            if token not in query.split():
                if token not in wordfreq.keys():
                    wordfreq[token] = 1
                else:
                    wordfreq[token] += 1

    d_descending = OrderedDict(sorted(wordfreq.items(), key=lambda kv: kv[1], reverse=True))
    word_list = list(d_descending.items())
    word_list
    return [{'text':word[0], 'value':word[1]} for word in word_list]

In [59]:
text = df_to_clean_text(df)

Text lenght:  6997197


In [80]:
query = 'porto '

In [81]:
query.split()

['porto']

In [82]:
"porto" not in query

False

In [60]:
wordfreq = word_freq(text,'todes','spanish')
wordfreq

[{'text': "''", 'value': 8978},
 {'text': '``', 'value': 8272},
 {'text': 'todas', 'value': 6108},
 {'text': 'ms', 'value': 5737},
 {'text': 'si', 'value': 5618},
 {'text': 'est', 'value': 4740},
 {'text': 'q', 'value': 3629},
 {'text': 'da', 'value': 3592},
 {'text': ')', 'value': 3549},
 {'text': 'ser', 'value': 2932},
 {'text': 'gracias', 'value': 2790},
 {'text': 'dice', 'value': 2746},
 {'text': 'as', 'value': 2505},
 {'text': 'nunca', 'value': 2378},
 {'text': 'va', 'value': 2222},
 {'text': 'hoy', 'value': 2103},
 {'text': 'gente', 'value': 2013},
 {'text': 'siempre', 'value': 1986},
 {'text': 'hace', 'value': 1842},
 {'text': 'alguien', 'value': 1791},
 {'text': 'das', 'value': 1727},
 {'text': 'rt', 'value': 1710},
 {'text': 'haciendo', 'value': 1696},
 {'text': 'aos', 'value': 1683},
 {'text': 'presidente', 'value': 1576},
 {'text': 'vos', 'value': 1573},
 {'text': 'menos', 'value': 1531},
 {'text': 'estn', 'value': 1525},
 {'text': 'hacen', 'value': 1517},
 {'text': 'alferde

In [56]:
# for tup in wordfreq:
#     print(tup)
word_list = list(wordfreq.items())
word_list
[{'text':word[0], 'value':word[1]} for word in word_list]

[{'text': ',', 'value': 44269},
 {'text': 'todes', 'value': 39324},
 {'text': 'a', 'value': 39055},
 {'text': 'que', 'value': 38848},
 {'text': 'de', 'value': 38489},
 {'text': '.', 'value': 32229},
 {'text': 'y', 'value': 32161},
 {'text': '@', 'value': 30646},
 {'text': 'la', 'value': 26156},
 {'text': 'el', 'value': 20692},
 {'text': '!', 'value': 17900},
 {'text': 'en', 'value': 17651},
 {'text': 'no', 'value': 16210},
 {'text': 'es', 'value': 13165},
 {'text': 'para', 'value': 12699},
 {'text': '#', 'value': 12440},
 {'text': 'con', 'value': 9476},
 {'text': 'los', 'value': 9470},
 {'text': 'por', 'value': 9370},
 {'text': 'un', 'value': 9339},
 {'text': '?', 'value': 9245},
 {'text': "''", 'value': 8687},
 {'text': 'lo', 'value': 8279},
 {'text': 'se', 'value': 7828},
 {'text': '``', 'value': 7148},
 {'text': 'las', 'value': 7084},
 {'text': 'todos', 'value': 6849},
 {'text': 'una', 'value': 6795},
 {'text': 'te', 'value': 5649},
 {'text': 'todas', 'value': 5623},
 {'text': 'del'

## Datacamp Course

#### Here I start with the datacamp course guide 👇🏼

In [82]:
vocabulary = sorted(set(text))

# testing erasing some parts of the vocabulary!
# vocabulary = vocabulary[16:60]

# Print the vocabulary size
print('Vocabulary size:', len(vocabulary))
vocabulary

Vocabulary size: 58


[' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|']

In [84]:
# Dictionary to save the mapping from char to integer
char_to_idx = { char : idx for idx, char in enumerate(vocabulary) }

# Dictionary to save the mapping from integer to char
idx_to_char = { idx : char for idx, char in enumerate(vocabulary) }

# Print char_to_idx and idx_to_char
print(char_to_idx)
print(idx_to_char)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, ')': 8, '*': 9, '+': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '_': 29, '`': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56, '|': 57}
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: ')', 9: '*', 10: '+', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '?', 28: '@', 29: '_', 30: '`', 31: 'a', 32: 'b', 33: 'c', 34: 'd', 35: 'e', 36: 'f', 37: 'g', 38: 'h', 39: 'i', 40: 'j', 41: 'k', 42: 'l', 43: 'm', 44: 'n', 45: 'o', 46: 'p', 47: 'q', 48: 'r', 49: 's', 50: 't', 51: 'u', 52: 'v', 53: 'w', 54: 'x', 5

In [85]:
maxlen =40

In [86]:
# Create empty lists for input and target dataset
input_data = []
target_data = []

# Iterate to get all substrings of length maxlen
for i in range(0, len(text) - maxlen):
    # Find the sequence of length maxlen starting at i
    input_data.append(text[i : i+maxlen])
    
    # Find the next char after this sequence 
    target_data.append(text[i+maxlen])

# Print number of sequences in input data
print('No of Sequences:', len(input_data))

No of Sequences: 207125


In [87]:
input_data

['vimos en prime time cmo sacaban a una mo',
 'imos en prime time cmo sacaban a una mom',
 'mos en prime time cmo sacaban a una momi',
 'os en prime time cmo sacaban a una momia',
 's en prime time cmo sacaban a una momia ',
 ' en prime time cmo sacaban a una momia p',
 'en prime time cmo sacaban a una momia po',
 'n prime time cmo sacaban a una momia por',
 ' prime time cmo sacaban a una momia por ',
 'prime time cmo sacaban a una momia por d',
 'rime time cmo sacaban a una momia por di',
 'ime time cmo sacaban a una momia por dig',
 'me time cmo sacaban a una momia por dign',
 'e time cmo sacaban a una momia por digni',
 ' time cmo sacaban a una momia por dignid',
 'time cmo sacaban a una momia por dignida',
 'ime cmo sacaban a una momia por dignidad',
 'me cmo sacaban a una momia por dignidad ',
 'e cmo sacaban a una momia por dignidad a',
 ' cmo sacaban a una momia por dignidad a ',
 'cmo sacaban a una momia por dignidad a l',
 'mo sacaban a una momia por dignidad a la',
 'o sacaba

In [88]:
# Create a 3-D zero vector to contain the encoded input sequences
x = np.zeros((len(input_data), maxlen, len(vocabulary)), dtype='float32')

# Create a 2-D zero vector to contain the encoded target characters
y = np.zeros((len(target_data), len(vocabulary)), dtype='float32')

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [89]:
# Iterate over the sequences
for s_idx, sequence in enumerate(input_data):
    # Iterate over all characters in the sequence
    for idx, char in enumerate(sequence):
        # Fill up vector x
        x[s_idx, idx, char_to_idx[char]] = 1    
    # Fill up vector y
    y[s_idx, char_to_idx[target_data[s_idx]]] = 1

In [90]:
# Create Sequential model 
model = tf.keras.Sequential()

# Add an LSTM layer of 128 units
model.add(layers.LSTM(128, input_shape=(maxlen, len(vocabulary))))

# Add a Dense output layer
model.add(layers.Dense(len(vocabulary), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               95744     
_________________________________________________________________
dense_2 (Dense)              (None, 58)                7482      
Total params: 103,226
Trainable params: 103,226
Non-trainable params: 0
_________________________________________________________________


In [91]:
# Fit the model
model.fit(x, y, batch_size=64, epochs=3, validation_split=0.2)

Epoch 1/3
2590/2590 [==============================] - 76s 29ms/step - loss: 2.4198 - val_loss: 2.1977
Epoch 2/3
2590/2590 [==============================] - 72s 28ms/step - loss: 2.1150 - val_loss: 2.0563
Epoch 3/3
2590/2590 [==============================] - 72s 28ms/step - loss: 1.9743 - val_loss: 1.9600


In [156]:
def generate_text(sentence, n):
    generated = sentence
#     generated += sentence
    for i in range(n):
    # Create a 3-D zero vector to contain the encoding of sentence.
        x_pred = np.zeros((1, maxlen, len(vocabulary)))
        
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_idx[char]] = 1.
            
        # Get probability distribution for the next character
        preds = model.predict(x_pred, verbose=0)[0]
        #print(preds)
    
        # Get the character with maximum probability
        next_index = np.argmax(preds)
        #next_index = random.sample(list(preds),0.7)
        next_index = random.choice(sorted(enumerate(example),
                       key=lambda x: x[1]
                       )[-15:])[0]
# write an if statement to check if most recent character generated is a space, if it is then generate randomly next character
# from -15 to -1 (from some array without array)

        next_char = idx_to_char[next_index]
    
        # Append the new character to the next input and generated text
        sentence = sentence[1:] + next_char
        generated += next_char
    
    # Print the generated text
    print(generated)


In [157]:
# Input sequence and generate text
sentence = "esa gente debe parar"
generate_text(sentence, 20)



esa gente debe pararrot eiesaipmcuerdupt


In [144]:
example

array([1.43619001e-01, 3.09601403e-03, 1.86012650e-03, 7.71214627e-03,
       9.98674586e-05, 7.28055093e-05, 6.76896743e-05, 1.16962634e-04,
       2.13351916e-04, 3.77779710e-04, 9.66791777e-05, 8.36510863e-03,
       8.72493314e-04, 5.97984344e-03, 1.41052855e-03, 1.49698963e-03,
       1.42622343e-03, 1.73081027e-03, 3.87489388e-04, 6.21355372e-04,
       3.60349281e-04, 2.86770461e-04, 9.16702498e-04, 3.28758266e-04,
       7.94419495e-04, 1.88012538e-03, 2.87854695e-04, 3.85484332e-03,
       1.22461161e-02, 2.62174057e-03, 1.74851903e-05, 1.22490332e-01,
       1.35361422e-02, 2.83862799e-02, 1.85262151e-02, 1.29976451e-01,
       3.90357804e-03, 8.26939382e-03, 1.49073042e-02, 1.20078430e-01,
       9.32709407e-03, 6.04929216e-03, 3.83987315e-02, 1.69543214e-02,
       2.12259181e-02, 5.12126982e-02, 2.96521783e-02, 6.18054438e-03,
       3.06836907e-02, 3.23542431e-02, 3.19511853e-02, 3.48651297e-02,
       9.62895993e-03, 2.83321552e-03, 2.35190894e-03, 1.06181921e-02,
      

In [149]:

y_hat = random.choice(sorted(enumerate(example),
                       key=lambda x: x[1]
                       )[-15:])[0]

In [150]:
y_hat

48